In [ ]:
import datetime as dt
import pandas as pd
import requests
import time
import urllib.request 
import re 

from bs4 import BeautifulSoup
from pandas import DataFrame
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [ ]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

In [ ]:
# 갤럭시S21
# https://search.shopping.naver.com/catalog/25604807522?query=%EA%B0%A4%EB%9F%AD%EC%8B%9C%2021&NaPm=ct%3Dkw5xr468%7Cci%3Dc8a7d7681596956d9cf8a2bfe6a6e38af8f0ab98%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Df185570a945b0d72d4fd341521e9777481191855

# 아이폰 12프로
# https://search.shopping.naver.com/catalog/24452731530?query=%EC%95%84%EC%9D%B4%ED%8F%B0%2012&NaPm=ct%3Dkw5xsc5k%7Cci%3Dea7fd4ce956f295640b808d46d2e1d5d9e23b3d2%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dfd7476f62d008e9ada804b4ea513cc0b8b08d197


# 갤럭시 S21

## 카테고리별 리뷰 수집

In [ ]:
name=['갤럭시S21','아이폰12프로']
category=['만족도','가격', '색상','디자인','사이즈','품질','사용성','제품구성','무게', '제품상태', '배터리','와이파이']

#z플립 검색
ns_address_galaxy="https://search.shopping.naver.com/catalog/25604807522?query=%EA%B0%A4%EB%9F%AD%EC%8B%9C%2021&NaPm=ct%3Dkw5xr468%7Cci%3Dc8a7d7681596956d9cf8a2bfe6a6e38af8f0ab98"

shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[4]/a/strong"
category1="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[2]/a" #만족도
category2="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[3]/a" #가격
category3="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[4]/a" #색상
category4="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[5]/a" #디자인
category5="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[6]/a"  #사이즈
category6="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[7]/a" #품질
category7="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[8]/a" #사용성
category8="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[9]/a" #제품구성
category9="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[10]/a" #무게
category10="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[11]/a" #제품상태
category11="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[12]/a" #배터리
category12="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[13]/a" #와이파이

header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

def save():
    if not os.path.exists('output1.csv'):
        df1.to_csv('output1.csv', encoding='utf-8-sig', mode='w')
    else:
        df1.to_csv('output1.csv',encoding='utf-8-sig', mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

        
#"만족도" 관련 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1

while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 

            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()



In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"가격" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()


In [ ]:

header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name_=name[0]  #z플립
category_=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()



In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#가격

d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name_=name[0]  
category_=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()


In [ ]:

header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name_=name[0]  
category_=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 

soup = BeautifulSoup(html, "html.parser")


d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category6).click() #스크롤 건드리면 안됨
name_=name[0]  
category_=category[5]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category7).click() #스크롤 건드리면 안됨
name_=name[0]  
category_=category[6]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category8).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[7]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category9).click() #스크롤 건드리면 안됨
name_=name[0]  
category_=category[8]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

###################해상도#######################
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category10).click() #스크롤 건드리면 안됨
name_=name[0]  
category_=category[9]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

###################해상도#######################
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category11).click() #스크롤 건드리면 안됨
name_=name[0]  
category_=category[10]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

###################해상도#######################
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category12).click() #스크롤 건드리면 안됨
name_=name[0]  
category_=category[11]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

## 전체 리뷰수집

In [ ]:
name=['갤럭시S21','아이폰12프로']
category=['만족도','가격', '색상','디자인','사이즈','품질','사용성','제품구성','무게', '제품상태', '배터리','와이파이']

#z플립 검색
ns_address_galaxy="https://search.shopping.naver.com/catalog/25604807522?query=%EA%B0%A4%EB%9F%AD%EC%8B%9C%2021&NaPm=ct%3Dkw5xr468%7Cci%3Dc8a7d7681596956d9cf8a2bfe6a6e38af8f0ab98"


shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[4]/a/strong"
total_category = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[1]/a"

header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_galaxy)
req = requests.get(ns_address_galaxy)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

def save():
    if not os.path.exists('output_galaxy_total.csv'):
        df1.to_csv('output_galaxy_total.csv', encoding='utf-8-sig', mode='w')
    else:
        df1.to_csv('output_galaxy_total.csv',encoding='utf-8-sig', mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

#전체 리뷰 가져오기
d.find_element_by_xpath(total_category).click() #스크롤 건드리면 안됨
name_=name[1] 
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1

while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        if page<100:
            try: #page11부터
                if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
                else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
                page+=1
            except: break 
        else:
            except: break 

            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()




# 아이폰

## 전체 리뷰 수집

In [ ]:
name=['갤럭시S21','아이폰12프로']
category=['만족도','가격', '색상','디자인','사이즈','품질','사용성','제품구성','무게', '제품상태', '배터리', '와이파이']

#아이폰12프로
ns_address_iphone="https://search.shopping.naver.com/catalog/24452731530?query=%EC%95%84%EC%9D%B4%ED%8F%B0%2012&NaPm=ct%3Dkw5xsc5k%7Cci%3Dea7fd4ce956f295640b808d46d2e1d5d9e23b3d2%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dfd7476f62d008e9ada804b4ea513cc0b8b08d197"

shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[4]/a/strong"
total_category = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[1]/a"

header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

def save():
    if not os.path.exists('output_iphone_total.csv'):
        df1.to_csv('output_iphone_total.csv', encoding='utf-8-sig', mode='w')
    else:
        df1.to_csv('output_iphone_total.csv',encoding='utf-8-sig', mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

#전체 리뷰 가져오기
d.find_element_by_xpath(total_category).click() #스크롤 건드리면 안됨
name_=name[1] 
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1

while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 

            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()



<ipython-input-31-e552dc0bd747>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-31-e552dc0bd747>:41: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-31-e552dc0bd747>:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(total_category).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-31-e552dc0bd747>:59: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-31-e552dc0bd747>:61: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
<ipython-input-31-e552dc0bd747>:64: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


1 아이폰을 사고 배송받은 뒤 며칠 사용해본 후에 리뷰 작성하고 싶어서 이제서야 후기를 씁니다ㅎㅎ🍎구매계기사전예약은 커녕 출시된 이후에도 재고 기다리는 사람들을 보고 이번 시리즈 사
아이폰을 사고 배송받은 뒤 며칠 사용해본 후에 리뷰 작성하고 싶어서 이제서야 후기를 씁니다ㅎㅎ

🍎구매계기
사전예약은 커녕 출시된 이후에도 재고 기다리는 사람들을 보고 이번 시리즈 사는 건 포기해야겠다고 생각했어요.카드 행사나 적립금 하나 없이 제 가격 다 주고 사는 건 싫고 핸드폰이 갖고 싶기는 하고 그래서 이리저리 서치하다가 찾은 것이 네이버 애플 공식 판매점입니다.공식이라는 설명에 마음이 갔고 여러 방법 다 하면 가격도 십만원 정도 싸게 살 수 있다는 것이 가장 .

🍎재고
제가 사려고 알아보던 때에는 모든 색상이 다 품절이었어요.사전예약도 아니고 조금 더 기다리면 언젠가는 들어오겠지라는 생각으로 매일 들어와서 재고확인 했던 것 같아요.오전 10시~11시쯤에 한번 물량 풀리고 오후 5시쯤에 또 한번 풀렸었는데 그파랑 실버,골드 다 들어오는데 퍼블만 재고가 안 들어와서 며칠 더 기다렸어요ㅠ지금은 모든 상품이 다 품절이 안되서 원활하게 구매하실 수 있으실 것 같네요!

🍎색상
퍼시픽 블루가 출시된다는 소식을 들었을 때부터 이번 핸드폰은 이거다 라고 생각했어요.근데 막상 구매까지 시간이 좀 걸리는 도중에 보니까 실버가 구하기 힘들다고 하니까 그것도 탐나서 이리저리 헤매다가 결국은 다시 블루로 돌아와 구매했습니다.구매하고 나서도 질리면 어쩌지 고민 많이 했는데 직접 보는 순간 그런 생각이 다 사라졌습니다..영상이랑 사진 전부 실물을 못 담아요..프로 자체가 무광이라 보는 각도마다 그파로 보이기도 하고 네이비로도 보였다가 블루로도 보여요!퍼블 사고 질리면 어떡하냐는 말을 제일 많이 들었는데 실버도 계속 실버만 보면 질리고 그파도 그파만 보면 질리듯이 똑같은 것 같습니다!

🍎배송
후기 보면서 배송이 엄청 늦었다, 타지역으로 상품이 가서 별로라는 글이 꽤 있었는데 제가 사는 지역까지 오

17 잘받았어요 택배강국 대한민국 택배기사님들 존경해요 엑스쓰다가 액정이 나가서 ㅠㅠ 오매불망 기다리다가 구매했어요 기존폰에서 바로 옮겨지는거 너무 좋구요 불량없어요 처음에 오줌액정인가
잘받았어요 택배강국 대한민국 택배기사님들 존경해요 엑스쓰다가 액정이 나가서 ㅠㅠ 오매불망 기다리다가 구매했어요 기존폰에서 바로 옮겨지는거 너무 좋구요 불량없어요 처음에 오줌액정인가했는데 인터넷 검색해서 잘 조정했어요 고장난 핸드폰에서 탈피 속이 시원해요 속도감이 확실히 빠른게
느껴져요 공식이라 믿음가고 저는 네이버 플러스멤버쉽으로 추가 혜택까지! 애프례어 할인 대박 감사해요~~~ 평점5 

18 너무 예뻐요ㅠㅠ
와 실버색상 열흘이상걸린다고 해서 이번주에 받을거라 기대도 안하고 있었는데 화욜날 주문해서 토욜날 받았어요!! 택배기사님도 환하게 웃으시면서 아이폰왓네요라고 저랑 같이 기뻐해주셨답니다ㅋㅋ 보호필름이랑 케이스등장 아무것도 주문안한상태라 그거받으면 만져보려고했는데 못참고 초기불량체크부터 데이터옮기는거 결국 셀프 확정기변등록까지 다 끝냈어요 불량하나없이 넘 만족스럽구요 지금 케이스 없는상태라 기스라도 생길까봐 상자에 넣어놓은 상태로 후기작성중이에요ㅋㅋ 빠른배송 불량없는상품 넘넘 감사합니다~^^ 평점5 

19 1.배송 배송 빠르게 받았습니다! 주문하고 주말이 껴있었지만 다음주에 안전하고 빠르게 도착하였습니다. 2. 포장 아무래도 전자부품인지라 어떻게 배송이 올지 고민 많았습니다ㅠㅠ걱정과
1.배송
배송 빠르게 받았습니다! 주문하고 주말이 껴있었지만 다음주에 안전하고 빠르게 도착하였습니다.
2. 포장
아무래도 전자부품인지라 어떻게 배송이 올지 고민 많았습니다ㅠㅠ걱정과 다르게 택배상자도 테이핑 되어 있고 안에도 뽁뽁이를 사용하여 안전하게 온 것 같습니다:)
3. 후기
진짜 🤭 품절이라서 수시로 들어와 확인했는데 혹시나 새벽에 들어가서 새로고침을 하자 바로 구매창이 열려서 구매했슴돠~! 저의 픽은 급지고 각은 어찌나 이쁜지.. 아이폰8에서 넘어가서 적응은 힘들었지만 너무 소중한 저의

...

1161 배송이 빠르진 않았지만 불량없는 걸로 잘 왔네요
배송이 빠르진 않았지만 불량없는 걸로 잘 왔네요 평점5 

1162 아이폰 8에서 넘어왔는데 아직까지는 만족합니다
아이폰 8에서 넘어왔는데 아직까지는 만족합니다 평점5 

1163 오래 기다렸고 잘 샀네요 이쁘긴 겁나 이뻐요. ㅠ
오래 기다렸고 잘 샀네요 이쁘긴 겁나 이뻐요. ㅠ 평점5 

1164 그래파이트 성공했어요
잘쓰고있
그래파이트 성공했어요
잘쓰고있습니다.
좋아요 평점5 

1165 만족도높고 싸게잘삿어요. 기능이많아서 좋음
만족도높고 싸게잘삿어요. 기능이많아서 좋음 평점5 

1166 배송이 좀 늦었지만 그래도 받으니까 좋네요
배송이 좀 늦었지만 그래도 받으니까 좋네요 평점5 

1167 색상으로 고민많았는데 넘 마음에 들어요 ^^
색상으로 고민많았는데 넘 마음에 들어요 ^^ 평점5 

1168 아이패드 프로 스그랑 비교샷입니
아이패드 프로 스그랑 비교샷입니다 ㅎㅎ 평점5 

1169 배송빨라서 바로 받았어요. 잘쓰겠습니다~
배송빨라서 바로 받았어요. 잘쓰겠습니다~ 평점5 

1170 역시 아이폰은 아이폰 ㅈㄴ이쁘네
역시 아이폰은 아이폰 ㅈㄴ이쁘네 평점5 

1171 그래파이트로 구매하길 잘 한 것
그래파이트로 구매하길 잘 한 것 같아요 평점5 

1172 와이프랑 하나씩 사서 잘 쓰고 있습니다
와이프랑 하나씩 사서 잘 쓰고 있습니다 평점5 

1173 배송도 빠르고 잘 사용 하고있습니다 !
배송도 빠르고 잘 사용 하고있습니다 ! 평점5 

1174 실버 뒷면 너무 영롱하고 이뻐요 ㅠㅠ 아무 탈없이 잘 전달받았고 꽤 빨리 받았어요!
실버 뒷면 너무 영롱하고 이뻐요 ㅠㅠ 아무 탈없이 잘 전달받았고 꽤 빨리 받았어요! 평점5 

1175 너무 좋아요.이미지보다 더 좋습니다.
너무 좋아요.
이미지보다 더 좋습니다. 평점5 

1176 포인트 포함해서 저렴하게 잘 샀네요
포인트 포함해서 저렴하게 잘 샀네요 평점5 

1177 배송빠르고 핸드폰 깬지 작살이네요
배송빠르고 핸드폰 깬지 작살이네요 평점5 

1281 좋아요 만족해요 잘쓸게요
좋아요 만족해요 잘쓸게요 평점5 

1282 배송도 빠르고 양품입니다
배송도 빠르고 양품입니다 평점5 

1283 아이폰 12 pro!!!! 좋습니다!
아이폰 12 pro!!!! 좋습니다! 평점5 

1284 제품은 깔끔 배송은 빠름
제품은 깔끔 배송은 빠름 평점5 

1285 배송이 진짜 빨랐습니다
배송이 진짜 빨랐습니다 평점5 

1286 안전하게 배송잘 왔네요
안전하게 배송잘 왔네요 평점5 

1287 빠른 배송으로 잘받았음
빠른 배송으로 잘받았음 평점5 

1288 양품받앗어요 빠른배송
양품받앗어요 빠른배송 평점5 

1289 역시 아이폰 최고에요 !
역시 아이폰 최고에요 ! 평점5 

1290 좀힘들지만 노력해야
좀힘들지만 노력해야 평점4 

1291 너무이쁩니다 크 ~~~~~
너무이쁩니다 크 ~~~~~ 평점5 

1292 일찍 잘 받았습니다.
일찍 잘 받았습니다. 평점5 

1293 빠른 배송 좋아요~~~~
빠른 배송 좋아요~~~~ 평점5 

1294 존예....너무 예뻐요
존예....너무 예뻐요 평점5 

1295 너무 좋습니다!!!!
너무 좋습니다!!!! 평점5 

1296 배송 빠르고 양품 받았습니다- 감사합니다
배송 빠르고 양품 받았습니다- 감사합니다 평점5 

1297 좋아요^^^^^^^^
좋아요^^^^^^^^ 평점5 

1298 30일에 발송하신다더니 실제로는 출시일에 받았어요 좋네요
30일에 발송하신다더니 실제로는 출시일에 받았어요 좋네요 평점5 

1299 대리점 사전예약하고 기다리다 지쳐서 자급제 찾아보다가.. 여기서 원하던 상품이 있어서 구매하고 이틀만이 배송와서 너무 좋았어요ㅠㅠ 안전하게 빠른배송 감사합니다~ 아이폰12프로 생각
대리점 사전예약하고 기다리다 지쳐서 자급제 찾아보다가.. 여기서 원하던 상품이 있어서 구매하고 이틀만이 배송와서 너무 좋았어요ㅠㅠ 안전하게 빠른배송 감사합니다~ 아이폰12프로 생각보다 더 이뻐서 좋아요🥰 평점5 

1300 1차예약분으로 샀어요. 출시 당일
1차예약분으로 

1408 배송 빠르고 양품이예요. 좋습니다 :)
배송 빠르고 양품이예요. 좋습니다 :) 평점5 

1409 만족합니다. 좋습니다. ㅋ
만족합니다. 좋습니다. ㅋ 평점5 

1410 꺄올!! 배송도 빨리오고 너무 좋아여
꺄올!! 배송도 빨리오고 너무 좋아여 평점5 

1411 드디어 7에서 벗어나고 구매했어요!
드디어 7에서 벗어나고 구매했어요! 평점5 

1412 배송이 정말 빠른것이 좋았습니다!
배송이 정말 빠른것이 좋았습니다! 평점5 

1413 제품 하자 없고 배송도 빠릅니다!
제품 하자 없고 배송도 빠릅니다! 평점5 

1414 배송도빠르고 포장도꼼꼼하네요
배송도빠르고 포장도꼼꼼하네요 평점4 

1415 불량없이 빠른배송으로왔어요~
불량없이 빠른배송으로왔어요~ 평점5 

1416 역시 아이폰 영롱하네용
역시 아이폰 영롱하네용 평점5 

1417 디자인도 예쁘고 좋아요
디자인도 예쁘고 좋아요 평점5 

1418 ㅠㅠ기다린 보람이 있어요 사진 화질 말도 안되게 너무좋아요 배송은 완쥰 칼배송!!!김사합니다 ㅎㅎ
ㅠㅠ기다린 보람이 있어요 사진 화질 말도 안되게 너무좋아요 배송은 완쥰 칼배송!!!김사합니다 ㅎㅎ 평점5 

1419 빠르고 좋습니다 굿긋
빠르고 좋습니다 굿긋 평점5 

1420 빠른 배송이 마음에 들었습니다 감사합니다
빠른 배송이 마음에 들었습니다 감사합니다 평점5 

페이지 72 

1421 아이폰 좋아요. ^^
아이폰 좋아요. ^^ 평점5 

1422 7 쓰다가 바꿨는데, 좋네요 특히 카메라!ㅎㅎ배송도 빠르고! 주문한 케이스가 배송지연으로 케이스 없이 쓰려니 불안불안 할 뿐...
7 쓰다가 바꿨는데, 좋네요 특히 카메라!ㅎㅎ배송도 빠르고! 주문한 케이스가 배송지연으로 케이스 없이 쓰려니 불안불안 할 뿐... 평점4 

1423 말해뭐해요~ ~~~~~
말해뭐해요~ ~~~~~ 평점5 

1424 잘쓰고 있어요...
잘쓰고 있어요... 평점5 

1425 딸이 너무 원해서 주문했어요생각보다 빨리 받았어요
딸이 너무 원해서 주문했어요
생각보다 빨리 받

1525 배송 빠르고 제품 확실합니다 아이폰12 재고가 없어서 고통받았는데 프리스비덕택에 이제 탈출하네요
배송 빠르고 제품 확실합니다 아이폰12 재고가 없어서 고통받았는데 프리스비덕택에 이제 탈출하네요 평점5 

1526 애플 스토어보다 빠르고 정확한 배송 감사합니다!!!!
애플 스토어보다 빠르고 정확한 배송 감사합니다!!!! 평점5 

1527 선물용으로 샀는데 배송도 빨리서 크리스마스 이브에 줬네요ㅋㅋㅋ 양품이고 좋아요
선물용으로 샀는데 배송도 빨리서 크리스마스 이브에 줬네요ㅋㅋㅋ 양품이고 좋아요 평점5 

1528 배송도 예상기간에 맞춰서 오고 초기 불량도 없는 것 같습니다. 폰은 역시나 예상했던대로 예쁘네요. 오래 기다렸던만큼 만족스럽습네요 :)
배송도 예상기간에 맞춰서 오고 초기 불량도 없는 것 같습니다. 폰은 역시나 예상했던대로 예쁘네요. 오래 기다렸던만큼 만족스럽습네요 :) 평점5 

1529 아이폰7 쓰다가 갈아타니까... 너무 신세계예요ㅋㅋㅋ소프트웨어는 크게 변한 건 없지만, 화면이 커지니 너무 좋네요.배송도 엄청 빨랐어요!!!!
아이폰7 쓰다가 갈아타니까... 너무 신세계예요ㅋㅋㅋ
소프트웨어는 크게 변한 건 없지만, 화면이 커지니 너무 좋네요.
배송도 엄청 빨랐어요!!!! 평점5 

1530 배송 3일품질 뽑기운 최고ㅎㅎ 만족합니다
배송 3일
품질 뽑기운 최고ㅎㅎ 만족합니다 평점5 

1531 굳굳숟ㅅㄷ굳숟국숟숟숟수구숟구두구구둑
굳굳숟ㅅㄷ굳숟국숟숟숟수구숟구두구구둑 평점5 

1532 역시아이폰 좋습니다 ㅎㅎ ㄹㅎ
역시아이폰 좋습니다 ㅎㅎ ㄹㅎ 평점5 

1533 쬐끔 아쉬운 배송이지만, 핸드폰은 매우 만족합니다불량없이 좋네요
쬐끔 아쉬운 배송이지만, 핸드폰은 매우 만족합니다
불량없이 좋네요 평점4 

1534 좋아요.
배송은 좀 걸렸는데 물건은 만족합니다. 애플사이트보다는 비싸요. 평점4 

1535 티다에서 한달넘게 기다리다가 또 한달정도 기다리라 그래서 취소하고 이 사이트에서 입고되자마자 구매했습니다 배송은 3일 걸렸네요진작에 알아둘걸 ㅎ

1637 외관은 양품이나 아직 화면 테스트는 못해봤네요 어렵게 받아서 기분은 좋아요
외관은 양품이나 아직 화면 테스트는 못해봤네요 어렵게 받아서 기분은 좋아요 평점4 

1638 1차에 구매를 했는데,,, 배송에 문
1차에 구매를 했는데,,, 배송에 문제가 생겨서 늦게 받았네요
잘 처리되어 제품은 잘 받았습니다,
수고하세요 평점5 

1639 1차예약분으로 샀어요. 출시 당일
1차예약분으로 샀어요. 출시 당일 바로 배송됐네요. 할인도 받아서 좋아요. 평점5 

1640 배송도 빨랐고 불량도 없고 제대
배송도 빨랐고 불량도 없고 제대로 잘 왔어요 싸게 잘샀네요ㅎ 하이마트짱 평점5 

페이지 83 

1641 배송도 빠르고 카드할인도 너무
배송도 빠르고 카드할인도 너무 좋았습니다
내년 아이폰 13에 또 만나요 평점5 

1642 11월 둘째주에 배송 예상 했는데
11월 둘째주에 배송 예상 했는데 생각보다 빨리 도착해서 넘 좋네용!! 평점4 

1643 예상보다 일찍 배송되었고, 제품
예상보다 일찍 배송되었고, 제품에 하자 없이 잘 받았습니다. 굿 평점5 

1644 빠른배송 좋았습니다. 아이폰은 두말할 필요없구요.
빠른배송 좋았습니다. 아이폰은 두말할 필요없구요. 평점5 

1645 너무 좋아유~
배송빠르게 잘받았습니다 그레파이트 넘예뻐유! 평점5 

1646 잘받았습니다 진짜로 돟네요 정상 작동되요
잘받았습니다 진짜로 돟네요 정상 작동되요 평점5 

1647 예상 배송일보다 빨리 왔어요
좋
예상 배송일보다 빨리 왔어요
좋아요🥳 평점5 

1648 끝까지
신경써주셔서 감사했습니다 잘쓸게요~~~^^ 평점5 

1649 정말 잘 받았습니다. 딸아이가 좋
정말 잘 받았습니다. 딸아이가 좋아해요. 평점5 

1650 손상없이 잘왔네요 생각보다 빨리왔어요
손상없이 잘왔네요 생각보다 빨리왔어요 평점4 

1651 엄청 빠른 배송과 좋은 품질 최고입니다
엄청 빠른 배송과 좋은 품질 최고입니다 평점5 

1652 진짜 이번에 아이폰 12프로 너무 예뻐요
진짜 이번에 아이폰 12

## 카테고리별 리뷰 수집

In [ ]:
name=['갤럭시S21','아이폰12프로']
category=['만족도','가격', '색상','디자인','사이즈','품질','사용성','제품구성','무게', '제품상태', '배터리','와이파이']


ns_address_iphone="https://search.shopping.naver.com/catalog/24452731530?query=%EC%95%84%EC%9D%B4%ED%8F%B0%2012&NaPm=ct%3Dkw5xsc5k%7Cci%3Dea7fd4ce956f295640b808d46d2e1d5d9e23b3d2%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dfd7476f62d008e9ada804b4ea513cc0b8b08d197"

shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[4]/a/strong"
category1="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[2]/a" #만족도
category2="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[3]/a" #가격
category3="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[5]/a" #색상
category4="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[4]/a" #디자인
category5="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[8]/a"  #사이즈
category6="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[7]/a" #품질
category7="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[11]/a" #사용성
category8="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[10]/a" #제품구성
category9="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[9]/a" #무게
category10="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[6]/a" #제품상태
category11="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[12]/a" #배터리
category12="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/div/ul/li[13]/a" #와이파이

header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

def save():
    if not os.path.exists('output_iphonecategory.csv'):
        df2.to_csv('output_iphonecategory.csv', encoding='utf-8-sig', mode='w')
    else:
        df2.to_csv('output_iphonecategory.csv',encoding='utf-8-sig', mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

        
#"만족도" 관련 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name_=name[1]  
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1

while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 

            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()



<ipython-input-18-cdd6a757311b>:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-18-cdd6a757311b>:52: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-18-cdd6a757311b>:57: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-18-cdd6a757311b>:71: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-18-cdd6a757311b>:73: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 아이폰을 사고 배송받은 뒤 며칠 사용해본 후에 리뷰 작성하고 싶어서 이제서야 후기를 씁니다ㅎㅎ🍎구매계기사전예약은 커녕 출시된 이후에도 재고 기다리는 사람들을 보고 이번 시리즈 사
아이폰을 사고 배송받은 뒤 며칠 사용해본 후에 리뷰 작성하고 싶어서 이제서야 후기를 씁니다ㅎㅎ

🍎구매계기
사전예약은 커녕 출시된 이후에도 재고 기다리는 사람들을 보고 이번 시리즈 사는 건 포기해야겠다고 생각했어요.카드 행사나 적립금 하나 없이 제 가격 다 주고 사는 건 싫고 핸드폰이 갖고 싶기는 하고 그래서 이리저리 서치하다가 찾은 것이 네이버 애플 공식 판매점입니다.공식이라는 설명에 마음이 갔고 여러 방법 다 하면 가격도 십만원 정도 싸게 살 수 있다는 것이 가장 .

🍎재고
제가 사려고 알아보던 때에는 모든 색상이 다 품절이었어요.사전예약도 아니고 조금 더 기다리면 언젠가는 들어오겠지라는 생각으로 매일 들어와서 재고확인 했던 것 같아요.오전 10시~11시쯤에 한번 물량 풀리고 오후 5시쯤에 또 한번 풀렸었는데 그파랑 실버,골드 다 들어오는데 퍼블만 재고가 안 들어와서 며칠 더 기다렸어요ㅠ지금은 모든 상품이 다 품절이 안되서 원활하게 구매하실 수 있으실 것 같네요!

🍎색상
퍼시픽 블루가 출시된다는 소식을 들었을 때부터 이번 핸드폰은 이거다 라고 생각했어요.근데 막상 구매까지 시간이 좀 걸리는 도중에 보니까 실버가 구하기 힘들다고 하니까 그것도 탐나서 이리저리 헤매다가 결국은 다시 블루로 돌아와 구매했습니다.구매하고 나서도 질리면 어쩌지 고민 많이 했는데 직접 보는 순간 그런 생각이 다 사라졌습니다..영상이랑 사진 전부 실물을 못 담아요..프로 자체가 무광이라 보는 각도마다 그파로 보이기도 하고 네이비로도 보였다가 블루로도 보여요!퍼블 사고 질리면 어떡하냐는 말을 제일 많이 들었는데 실버도 계속 실버만 보면 질리고 그파도 그파만 보면 질리듯이 똑같은 것 같습니다!

🍎배송
후기 보면서 배송이 엄청 늦었다, 타지역으로 상품이 가서 별로라는 글이 꽤 있었는데 제가 사는 지역까지 오

<ipython-input-18-cdd6a757311b>:76: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


...

페이지 12 

221 거의 3차 사전예약때 주문 한거라 완전 늦게올줄 알고 케이스도 주문 안하고있었는데 빨리와서 깜짝 놀랐어요. 자급제는 처음인데 폰도 예쁘고 좋아요. 근데 화면이 다른 아이폰 화면
거의 3차 사전예약때 주문 한거라 완전 늦게올줄 알고 케이스도 주문 안하고있었는데 빨리와서 깜짝 놀랐어요. 자급제는 처음인데 폰도 예쁘고 좋아요. 근데 화면이 다른 아이폰 화면 보다 진짜 노란색이에요.. 애플에서 일부러 그렇게 만들었다고 하긴하던데.. ㅠ 그래도 애플케어 할인 받고 액정필름까지 주셔서 너무 좋아욜!! 윌리스몰 흥하세요!! 평점5 

222 마음에 듭니다. 박스자체도 탄탄해서 안전하게 왔구요. 불량테스트 해봣는데 문제 없네요. 위쪽 볼륨버튼이 조금 이상하긴 한데 문제는 없어서 괜춘~~ 케이스랑 필름 산게 빨리와야 사
마음에 듭니다. 박스자체도 탄탄해서 안전하게 왔구요. 불량테스트 해봣는데 문제 없네요. 위쪽 볼륨버튼이 조금 이상하긴 한데 문제는 없어서 괜춘~~ 케이스랑 필름 산게 빨리와야 사용할텐데... !! 아이폰 8 쓰다가 옮겨서 그런지 크게 감흥은 없지만..ㅎㅎㅎ 새거 라는 거가 기분이 좋네요~ 평점5 

223 진짜 한달을 원하는 재고 나오길 들어갔다 나갔다 기다렸는데 갑자기 재고가 풀려서 구매했습니당~ 혹시나 불량일까 걱정했는데 다행히 아직은 괜찮은거 같구요! 배송은 주말껴서 금요일에
진짜 한달을 원하는 재고 나오길 들어갔다 나갔다 기다렸는데 갑자기 재고가 풀려서 구매했습니당~ 혹시나 불량일까 걱정했는데 다행히 아직은 괜찮은거 같구요! 배송은 주말껴서 금요일에 주문하고 화요일에 받았어요~ 액정이 노랗게 오줌액정이란 말이 많았는데 저도 켜자마자 그랬는데 트루톤 끄니 괜찮아요~ 평점5 

224 배송은..음.. 충전기는 진짜 빨리왔는데 핸드폰은 일하는 저를 대신해서 전국일주를 하고 왔어요... 그래도 기다린보람이 있어요 너무너무 예쁘거든요 저는 일월에 거의 한달을 기다리고
배송은..음.. 충전기는 진짜 빨리왔는데 핸드폰은 일하는 저를 대신해서 

265 배송 빨라서 좋았어요! 출시 당일
배송 빨라서 좋았어요! 출시 당일 오전중에 도착했어요!
정말 예쁜 그래파이트입니다! 외관상태 깔끔했고, 디스플레이 불량도 없었습니다! 사전예약 하면서 카드할인으로 다소 저렴한 가격에 구입해서 대만족합니당~~ 평점5 

266 ㅂㅐ송도 빠른편이고 제품도 좋아요 색 존예
ㅂㅐ송도 빠른편이고 제품도 좋아요 색 존예 평점5 

267 배송빠르고 답변 빠르고 기기적응중 매우만족!
배송빠르고 답변 빠르고 기기적응중 매우만족! 평점5 

268 영홍하고 너무 예뻐요 상태도 좋고 네이버에서 구입해서 포인트도 짱 많이 받았어여 ☺️애지중지하며 잘 사용하고 있어요
영홍하고 너무 예뻐요 상태도 좋고 네이버에서 구입해서 포인트도 짱 많이 받았어여 ☺️애지중지하며 잘 사용하고 있어요 평점5 

269 연휴끼여서 좀 기다렸지만 잘 받았어요. 색상 고민 많이 했는데 골드 넘 예뻐요
연휴끼여서 좀 기다렸지만 잘 받았어요. 색상 고민 많이 했는데 골드 넘 예뻐요 평점5 

270 와이프 선물해줬는데 아주 좋아합니다. 아이폰8 오래 쓰다가 12로 갔는데 화면도 크고 아이폰 카메라 성능때문에 쓰는 것 같습니다~
와이프 선물해줬는데 아주 좋아합니다. 아이폰8 오래 쓰다가 12로 갔는데 화면도 크고 아이폰 카메라 성능때문에 쓰는 것 같습니다~ 평점5 

271 제품 잘 받았습니다
중간에 배송
제품 잘 받았습니다
중간에 배송에 문제가 생겨서 늦게 받았지만
제품에는 문제가 없어 다행입니다.
수고하세요 평점5 

272 오래기다린만큼 넘 예쁘네요 . 골드색상 너무 진하지도 않고 은은하니 고급스러움...
오래기다린만큼 넘 예쁘네요 . 골드색상 너무 진하지도 않고 은은하니 고급스러움... 평점5 

273 배송도 빠르고, 색상도 너무 이뻐요~대만족 입니다^^
배송도 빠르고, 색상도 너무 이뻐요~
대만족 입니다^^ 평점5 

274 배송이 빨라서 너무 좋았어요. 제
배송이 빨라서 너무 좋았어요. 제품도 완전 깨끗하고 안전하게 포장되어서 왔구요. 불량 없이 잘 도착했습

374 최고예요
믿고 구매하셔도 됩니다. 굿입니다요 평점5 

375 허브에 갇혀 좀 늦게 받았지만 양품으로 받아 바로 사용 했어요!! 애플케어 등록도 늦게 할 수 있음 좋을텐데 바로 개시 되는 부분만 좀 아쉽네요 ㅠ
허브에 갇혀 좀 늦게 받았지만 양품으로 받아 바로 사용 했어요!! 애플케어 등록도 늦게 할 수 있음 좋을텐데 바로 개시 되는 부분만 좀 아쉽네요 ㅠ 평점5 

376 최고예요
배송 엄청빠르네요 만족합니다 평점5 

377 좋아요
잘 쓰고 있어요.가격도 카드 할인 받아 만족하고요.자급제가 편한거 같아요. 평점4 

378 엄청 오랫동안 기다렸다가 드디어 성공했어요 ㅠㅠ실버 실물 너무 예뻐요!!
엄청 오랫동안 기다렸다가 드디어 성공했어요 ㅠㅠ
실버 실물 너무 예뻐요!! 평점5 

379 최고예요
배송도 빠르고 하자 없이 도착했어요너무 만족스럽게 사용하고 있습니다 평점5 

380 월요일 오전에 주문했는데 금요일에 받았네요 배송이 상당히 느려서 여유있게 주문하시는게 좋을듯 합니다 다행이 물건은 좋은듯 하네요
월요일 오전에 주문했는데 금요일에 받았네요 배송이 상당히 느려서 여유있게 주문하시는게 좋을듯 합니다 다행이 물건은 좋은듯 하네요 평점3 

페이지 20 

381 좋아요 배송은 3일만에 도착했어요 색상 이뻐요 충전단자 없어서 깜놀 했어요 ㅋㅋ
좋아요 배송은 3일만에 도착했어요 색상 이뻐요 충전단자 없어서 깜놀 했어요 ㅋㅋ 평점5 

382 재입고되길 오래 기다렸는데 잘 받았습니다. 예뻐요.
재입고되길 오래 기다렸는데 잘 받았습니다. 예뻐요. 평점5 

383 좋아요 배송도 빠르고 좋았어요 색도 넘 이쁘네요
좋아요 배송도 빠르고 좋았어요 색도 넘 이쁘네요 평점4 

384 예쁘고 좋아요배송도 주말에 사서 월요일 출발 화요일 도착아주 빨랐습니댜 :)
예쁘고 좋아요
배송도 주말에 사서 월요일 출발 화요일 도착
아주 빨랐습니댜 :) 평점5 

385 안전하게 잘 왔습니다. 너무 좋네요
안전하게 잘 왔습니다. 너무 좋네요 평점5 

386 골드 넘 예뻐요 

<ipython-input-18-cdd6a757311b>:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()


페이지 21 

401 남자친구가 진짜 좋아하네요ㅋㅋ
남자친구가 진짜 좋아하네요ㅋㅋ 평점5 

402 너무 좋습니다!!!!
너무 좋습니다!!!! 평점5 

403 대리점 사전예약하고 기다리다 지쳐서 자급제 찾아보다가.. 여기서 원하던 상품이 있어서 구매하고 이틀만이 배송와서 너무 좋았어요ㅠㅠ 안전하게 빠른배송 감사합니다~ 아이폰12프로 생각
대리점 사전예약하고 기다리다 지쳐서 자급제 찾아보다가.. 여기서 원하던 상품이 있어서 구매하고 이틀만이 배송와서 너무 좋았어요ㅠㅠ 안전하게 빠른배송 감사합니다~ 아이폰12프로 생각보다 더 이뻐서 좋아요🥰 평점5 

404 아이뻐 12 프로 역시 아이뻐. 자급제라 기존 요즘제 그대로 쓰고 좋네요
아이뻐 12 프로

역시 아이뻐.

자급제라 기존 요즘제 그대로 쓰고 좋네요 평점5 

405 모델부터성능까지모두모두좋아요아들이좋아하네요
모델부터성능까지모두모두좋아요
아들이좋아하네요 평점5 

406 사용해 보니 너무 너무 좋네요.. 추천드려요
사용해 보니 너무 너무 좋네요.. 추천드려요 평점5 

407 아주 마음에 들어요- 색상도 이쁘고 오줌액정이라고 보였는데 설정바꿔서 잘 쓰고 있습닏다
아주 마음에 들어요- 색상도 이쁘고
오줌액정이라고 보였는데 설정바꿔서 잘 쓰고 있습닏다 평점5 

408 최고예요
좋은 제품 잘 받았습니다. 설명도 친절해서 좋았습니다. 대박나세요~ 평점5 

409 역시 애플입니당 ㅠ 화질 미쳤고 핸드폰 넘나 조아용
역시 애플입니당 ㅠ 화질 미쳤고 핸드폰 넘나 조아용 평점5 

410 배송 완전 빠르고!! 너무 이뻐요ㅜㅜ 감사합니다
배송 완전 빠르고!! 너무 이뻐요ㅜㅜ 감사합니다 평점5 

411 오 배송도 빠르고 너무 잘 쓰고 있어요
오 배송도 빠르고 너무 잘 쓰고 있어요 평점5 

412 배송도 빠르고 상품도 만족합니다
배송도 빠르고 상품도 만족합니다. 평점5 

413 디자인은 역시 보면 볼수록 예쁩니다. 사용하는게 익숙해져서 편리합니다.
디자인은 역시 보면 볼수록 예쁩니다. 사용하는게 익숙해져서 편리합니

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name_=name[1]  #z플립
category_=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-19-0f9cac550620>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-19-0f9cac550620>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-19-0f9cac550620>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-19-0f9cac550620>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-19-0f9cac550620>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 잘받았어요 택배강국 대한민국 택배기사님들 존경해요 엑스쓰다가 액정이 나가서 ㅠㅠ 오매불망 기다리다가 구매했어요 기존폰에서 바로 옮겨지는거 너무 좋구요 불량없어요 처음에 오줌액정인가
잘받았어요 택배강국 대한민국 택배기사님들 존경해요 엑스쓰다가 액정이 나가서 ㅠㅠ 오매불망 기다리다가 구매했어요 기존폰에서 바로 옮겨지는거 너무 좋구요 불량없어요 처음에 오줌액정인가했는데 인터넷 검색해서 잘 조정했어요 고장난 핸드폰에서 탈피 속이 시원해요 속도감이 확실히 빠른게
느껴져요 공식이라 믿음가고 저는 네이버 플러스멤버쉽으로 추가 혜택까지! 애프례어 할인 대박 감사해요~~~ 평점5 



<ipython-input-19-0f9cac550620>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


2 아이폰12프로실버!!! 진짜 완전 예뻐욥 ㅜ.ㅜ! 일주일쯤 재고 언제 풀리나 들락날락하다가 갑자기 금욜인가 풀려서 오후 2시반쯤에 주문했는데 토요일 오전에 배송왔어요 ㅋㅋ 인천에서
아이폰12프로실버!!! 진짜 완전 예뻐욥 ㅜ.ㅜ! 일주일쯤 재고 언제 풀리나 들락날락하다가 갑자기 금욜인가 풀려서 오후 2시반쯤에 주문했는데 토요일 오전에 배송왔어요 ㅋㅋ 인천에서 배송시작인듯한데 제가 인천살아서 좀 더 빨리온것같기도하네요!! 아무튼 너무좋고요~ 공홈보다 싸게사서 더 좋아요 ㅋㅋㅋ 양품 잘 온 것 같습니다(아직까지는..!)ㅎㅎ 여러분 고민은 배송만 늦출뿐..! 지르십시오 ㅋㅋㅋ 평점5 

3 배송도 이틀 뒤에 바로 오고 양품인거 같았으나 처음 아이폰을 껏다가 키면 애플 로고가 켜졌다가 금방 사라지네요. 이거는 뭐 이용에 큰 문제가 없어서 지장은 없지만 사용 후 일주일
배송도 이틀 뒤에 바로 오고 양품인거 같았으나 처음 아이폰을 껏다가 키면 애플 로고가 켜졌다가 금방 사라지네요. 이거는 뭐 이용에 큰 문제가 없어서 지장은 없지만 사용 후 일주일 뒤에 갑자기 핸드폰이 멈추면서 재부팅이 됐네요(Panic-base) 아직 한번 밖에 되질 않아서 좀 지켜봐야 할거 같아요. 그것만 빼면 아주 빠릿빠릿하고 이쁜 핸드폰입니다! 다른 사이트와 비교해보면 자급제로 구매하는것이 이곳에서 사는게 가장 저렴하게 살 수 있는 곳이네요! 평점4 

4 아이폰12 pro 넘 구매하고 싶었는데 자급제라 ㅋㅍ에서 대기하다 매번 놓쳐서 인터넷으로 알아보던 중 애플 공식 판매점이 네이버 스마트스토어에 있어 우왕!하고 재고 확인해보니 있어
아이폰12 pro 넘 구매하고 싶었는데 자급제라 ㅋㅍ에서 대기하다 매번 놓쳐서 인터넷으로 알아보던 중 애플 공식 판매점이 네이버 스마트스토어에 있어 우왕!하고 재고 확인해보니 있어서 망설임없이 바로 구매했습니다 ㅠ 저는 네이버 플러스 멤버십이어서 적립을 많이 받아 조금 더 저렴하게 구매해 현재까지 잘 사용중이랍니다♡ 넘 좋아요옹~ 평점5 

5 실버가 실물이 너무너

<ipython-input-19-0f9cac550620>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


페이지 2 

21 잘받았어요 택배강국 대한민국 택배기사님들 존경해요 엑스쓰다가 액정이 나가서 ㅠㅠ 오매불망 기다리다가 구매했어요 기존폰에서 바로 옮겨지는거 너무 좋구요 불량없어요 처음에 오줌액정인가
잘받았어요 택배강국 대한민국 택배기사님들 존경해요 엑스쓰다가 액정이 나가서 ㅠㅠ 오매불망 기다리다가 구매했어요 기존폰에서 바로 옮겨지는거 너무 좋구요 불량없어요 처음에 오줌액정인가했는데 인터넷 검색해서 잘 조정했어요 고장난 핸드폰에서 탈피 속이 시원해요 속도감이 확실히 빠른게
느껴져요 공식이라 믿음가고 저는 네이버 플러스멤버쉽으로 추가 혜택까지! 애프례어 할인 대박 감사해요~~~ 평점5 

22 아이폰12프로실버!!! 진짜 완전 예뻐욥 ㅜ.ㅜ! 일주일쯤 재고 언제 풀리나 들락날락하다가 갑자기 금욜인가 풀려서 오후 2시반쯤에 주문했는데 토요일 오전에 배송왔어요 ㅋㅋ 인천에서
아이폰12프로실버!!! 진짜 완전 예뻐욥 ㅜ.ㅜ! 일주일쯤 재고 언제 풀리나 들락날락하다가 갑자기 금욜인가 풀려서 오후 2시반쯤에 주문했는데 토요일 오전에 배송왔어요 ㅋㅋ 인천에서 배송시작인듯한데 제가 인천살아서 좀 더 빨리온것같기도하네요!! 아무튼 너무좋고요~ 공홈보다 싸게사서 더 좋아요 ㅋㅋㅋ 양품 잘 온 것 같습니다(아직까지는..!)ㅎㅎ 여러분 고민은 배송만 늦출뿐..! 지르십시오 ㅋㅋㅋ 평점5 

23 배송도 이틀 뒤에 바로 오고 양품인거 같았으나 처음 아이폰을 껏다가 키면 애플 로고가 켜졌다가 금방 사라지네요. 이거는 뭐 이용에 큰 문제가 없어서 지장은 없지만 사용 후 일주일
배송도 이틀 뒤에 바로 오고 양품인거 같았으나 처음 아이폰을 껏다가 키면 애플 로고가 켜졌다가 금방 사라지네요. 이거는 뭐 이용에 큰 문제가 없어서 지장은 없지만 사용 후 일주일 뒤에 갑자기 핸드폰이 멈추면서 재부팅이 됐네요(Panic-base) 아직 한번 밖에 되질 않아서 좀 지켜봐야 할거 같아요. 그것만 빼면 아주 빠릿빠릿하고 이쁜 핸드폰입니다! 다른 사이트와 비교해보면 자급제로 구매하는것이 이곳에서 사는게 가

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name_=name[1]  #z플립
category_=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-20-6ee6d9c7c75b>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-20-6ee6d9c7c75b>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-20-6ee6d9c7c75b>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-20-6ee6d9c7c75b>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-20-6ee6d9c7c75b>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 기변 예약을 하고 대기타다가 우연히 크리스마스날 밤에 이 곳을 알게되어 셀프 선물로 바로 주문, 결제했습니다! 자급제는 처음이라 설레는 긴장으로 기다렸던 기억이 새록새록하네요!!휴
기변 예약을 하고 대기타다가 우연히 크리스마스날 밤에 이 곳을 알게되어 셀프 선물로 바로 주문, 결제했습니다! 자급제는 처음이라 설레는 긴장으로 기다렸던 기억이 새록새록하네요!!

휴일과 주말 포함 25일 결제, 30일 수령했습니다. 받고 보니 무엇보다 네이버페이로 다른 자급제보다 혜택이 큰 거 같아요:)

🤳🏻사용감
6s에서 넘어왔는데 모든 게 다 새롭긴 하지만 확실히 램이 늘어나서 빠릿빠릿한게 짜릿합니다... 아이폰 쓰신 분들은 아시겠지만 창 넘길 때 부드러운 거 못 잃는데 속도도 빠르고

🤳🏻컬러
처음부터 퍼시픽블루가 원픽이었는데 고민 끝에 라스트픽도 퍼블입니다🧜🏻‍♀️ 받는 빛의 양에 따라서 블루 빛이 달라보인다는 후기가 기억나네요! 빛의 양에 따라

🤳🏻사진
사실 제일 기대했던 부분인데 역시는 역시👏🏻
조명이나 해를 마주하고 찍으면 고스트 현상은 여전히 있더라구요! 첨부한 사진 참고하시면 같은 자리에서 확대 정도만 다르게 해서 찍었습니다! 어두운 데서도 잘 나오는게 굉장히 🥰 평점5 



<ipython-input-20-6ee6d9c7c75b>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


2 1.배송 배송 빠르게 받았습니다! 주문하고 주말이 껴있었지만 다음주에 안전하고 빠르게 도착하였습니다. 2. 포장 아무래도 전자부품인지라 어떻게 배송이 올지 고민 많았습니다ㅠㅠ걱정과
1.배송
배송 빠르게 받았습니다! 주문하고 주말이 껴있었지만 다음주에 안전하고 빠르게 도착하였습니다.
2. 포장
아무래도 전자부품인지라 어떻게 배송이 올지 고민 많았습니다ㅠㅠ걱정과 다르게 택배상자도 테이핑 되어 있고 안에도 뽁뽁이를 사용하여 안전하게 온 것 같습니다:)
3. 후기
진짜 🤭 품절이라서 수시로 들어와 확인했는데 혹시나 새벽에 들어가서 새로고침을 하자 바로 구매창이 열려서 구매했슴돠~! 저의 픽은 급지고 각은 어찌나 이쁜지.. 아이폰8에서 넘어가서 적응은 힘들었지만 너무 소중한 저의 아이폰이 되었슴돠🙏🏻💕 불량테스트했는데 불량 없고 정상입니다. 아이폰12 야간모드 너무 대박입니다..👍🏻모두 아이폰12프로 하세요🍎 평점5 

3 역시 ㅠㅠㅠ 너무 이쁩니다실버와 그래파이트 둘중 계속 고민하고 주문했다 취소하고를 반복한 끝에 그래파이트로 결정하고 주문했는데 너무 잘한 선택인거같아요무광이 너무 고급지고 이쁩니다
역시 ㅠㅠㅠ 너무 이쁩니다
실버와 그래파이트 둘중 계속 고민하고 주문했다 취소하고를 반복한 끝에 그래파이트로 결정하고 주문했는데 너무 잘한 선택인거같아요
무광이 너무 고급지고 이쁩니다 ㅠㅠ
512기가를 기다리다가 도저히 재고가 안나서 256기가로 주문했는데 만족해요!
5s를 썼을때 디자인 너무 만족하며 썼었던지라
이번 12프로가 나왔을때 너무이쁘다고 생각해서 사용하던 지플립을 버리고 다시 아이폰으로 넘어왔어요
카메라안에 먼지가 있는 불량을 받았는데 서비스센터에서 교환을 받고 양품사용중입니다 ㅎㅎ 평점5 

4 아이폰6-8플러스-12프로 짝수에디션 사용자가 됐네요ㅎㅎ 8플러스 아직 너무 멀쩡하고 잘 쓰고 있었는데 크기가 너무 컸어서 갈아타게 됐습니다ㅎㅎ 그립톡 없이도 폰을 들 수 있다는게
아이폰6-8플러스-12프로 짝수에디션 사용자가 됐네요ㅎㅎ 8플러스 아직 너무 멀쩡하

<ipython-input-20-6ee6d9c7c75b>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


페이지 2 

21 기변 예약을 하고 대기타다가 우연히 크리스마스날 밤에 이 곳을 알게되어 셀프 선물로 바로 주문, 결제했습니다! 자급제는 처음이라 설레는 긴장으로 기다렸던 기억이 새록새록하네요!!휴
기변 예약을 하고 대기타다가 우연히 크리스마스날 밤에 이 곳을 알게되어 셀프 선물로 바로 주문, 결제했습니다! 자급제는 처음이라 설레는 긴장으로 기다렸던 기억이 새록새록하네요!!

휴일과 주말 포함 25일 결제, 30일 수령했습니다. 받고 보니 무엇보다 네이버페이로 다른 자급제보다 혜택이 큰 거 같아요:)

🤳🏻사용감
6s에서 넘어왔는데 모든 게 다 새롭긴 하지만 확실히 램이 늘어나서 빠릿빠릿한게 짜릿합니다... 아이폰 쓰신 분들은 아시겠지만 창 넘길 때 부드러운 거 못 잃는데 속도도 빠르고

🤳🏻컬러
처음부터 퍼시픽블루가 원픽이었는데 고민 끝에 라스트픽도 퍼블입니다🧜🏻‍♀️ 받는 빛의 양에 따라서 블루 빛이 달라보인다는 후기가 기억나네요! 빛의 양에 따라

🤳🏻사진
사실 제일 기대했던 부분인데 역시는 역시👏🏻
조명이나 해를 마주하고 찍으면 고스트 현상은 여전히 있더라구요! 첨부한 사진 참고하시면 같은 자리에서 확대 정도만 다르게 해서 찍었습니다! 어두운 데서도 잘 나오는게 굉장히 🥰 평점5 

22 1.배송 배송 빠르게 받았습니다! 주문하고 주말이 껴있었지만 다음주에 안전하고 빠르게 도착하였습니다. 2. 포장 아무래도 전자부품인지라 어떻게 배송이 올지 고민 많았습니다ㅠㅠ걱정과
1.배송
배송 빠르게 받았습니다! 주문하고 주말이 껴있었지만 다음주에 안전하고 빠르게 도착하였습니다.
2. 포장
아무래도 전자부품인지라 어떻게 배송이 올지 고민 많았습니다ㅠㅠ걱정과 다르게 택배상자도 테이핑 되어 있고 안에도 뽁뽁이를 사용하여 안전하게 온 것 같습니다:)
3. 후기
진짜 🤭 품절이라서 수시로 들어와 확인했는데 혹시나 새벽에 들어가서 새로고침을 하자 바로 구매창이 열려서 구매했슴돠~! 저의 픽은 급지고 각은 어찌나 이쁜지.. 아이폰8에서 넘어가서 적응은 힘들었지만 너무 소중한 

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name_=name[1]  #z플
category_=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-21-f124c4fc7899>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-21-f124c4fc7899>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-21-f124c4fc7899>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-21-f124c4fc7899>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-21-f124c4fc7899>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 아이폰을 사고 배송받은 뒤 며칠 사용해본 후에 리뷰 작성하고 싶어서 이제서야 후기를 씁니다ㅎㅎ🍎구매계기사전예약은 커녕 출시된 이후에도 재고 기다리는 사람들을 보고 이번 시리즈 사
아이폰을 사고 배송받은 뒤 며칠 사용해본 후에 리뷰 작성하고 싶어서 이제서야 후기를 씁니다ㅎㅎ

🍎구매계기
사전예약은 커녕 출시된 이후에도 재고 기다리는 사람들을 보고 이번 시리즈 사는 건 포기해야겠다고 생각했어요.카드 행사나 적립금 하나 없이 제 가격 다 주고 사는 건 싫고 핸드폰이 갖고 싶기는 하고 그래서 이리저리 서치하다가 찾은 것이 네이버 애플 공식 판매점입니다.공식이라는 설명에 마음이 갔고 여러 방법 다 하면 가격도 십만원 정도 싸게 살 수 있다는 것이 가장 .

🍎재고
제가 사려고 알아보던 때에는 모든 색상이 다 품절이었어요.사전예약도 아니고 조금 더 기다리면 언젠가는 들어오겠지라는 생각으로 매일 들어와서 재고확인 했던 것 같아요.오전 10시~11시쯤에 한번 물량 풀리고 오후 5시쯤에 또 한번 풀렸었는데 그파랑 실버,골드 다 들어오는데 퍼블만 재고가 안 들어와서 며칠 더 기다렸어요ㅠ지금은 모든 상품이 다 품절이 안되서 원활하게 구매하실 수 있으실 것 같네요!

🍎색상
퍼시픽 블루가 출시된다는 소식을 들었을 때부터 이번 핸드폰은 이거다 라고 생각했어요.근데 막상 구매까지 시간이 좀 걸리는 도중에 보니까 실버가 구하기 힘들다고 하니까 그것도 탐나서 이리저리 헤매다가 결국은 다시 블루로 돌아와 구매했습니다.구매하고 나서도 질리면 어쩌지 고민 많이 했는데 직접 보는 순간 그런 생각이 다 사라졌습니다..영상이랑 사진 전부 실물을 못 담아요..프로 자체가 무광이라 보는 각도마다 그파로 보이기도 하고 네이비로도 보였다가 블루로도 보여요!퍼블 사고 질리면 어떡하냐는 말을 제일 많이 들었는데 실버도 계속 실버만 보면 질리고 그파도 그파만 보면 질리듯이 똑같은 것 같습니다!

🍎배송
후기 보면서 배송이 엄청 늦었다, 타지역으로 상품이 가서 별로라는 글이 꽤 있었는데 제가 사는 지역까지 오

<ipython-input-21-f124c4fc7899>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


2 아이폰 프로 12 실버 구매자 입니다^^기다리고 기다리던 제품을 빠르게 받아 볼 수 있었어요역시 뭐니뭐니해도 애플은 공식 홈페이지가 최고입니다!특히 자주 이용하는 네이버 스토어라
아이폰 프로 12 실버 구매자 입니다^^
기다리고 기다리던 제품을 빠르게 받아 볼 수 있었어요
역시 뭐니뭐니해도 애플은 공식 홈페이지가 최고입니다!
특히 자주 이용하는 네이버 스토어라 더욱 좋았구요
믿음이 갔습니다
당시 구하기 어렵던 아이폰 프로 였는데 남들보다 2개월은 먼저 사용 할 수 있었던 것 같아요
사용 중에도 전혀 기계적인 문제는 없었네요
공홈에서 사서 그런가 뽑기도 잘 된 느낌이에요
정말 😍
앞으로 몇년을 함께 해야 해서 어디서 사면 좋을 지 고민이 많았는데 ~
같이
역시 애플은 애플입니다
!!^_^ 평점5 

3 아이폰 초창기부터 계속 아이폰만 써왔어서 삼성폰은 손에 익지 않아서 계속 아이폰만 써오다 새로 나온 12가 예전 4모델처럼 각진 디자인으로 바뀌었다는 소리를 듣고 출시만 기다리고
아이폰 초창기부터 계속 아이폰만 써왔어서 삼성폰은 손에 익지 않아서 계속 아이폰만 써오다 새로 나온 12가 예전 4모델처럼 각진 디자인으로 바뀌었다는 소리를 듣고 출시만 기다리고 있다가 구매했어요~ 예약구매 놓치고 정가 다 주고 구매했지만 정말 후회 없네요~ 4모델의 약간은 투박하지만 질리지 않고 견고해보이는 다자인을 좋아했던 저로서는 정말 12모델이 흠잡을데 없이 딱 제 맘에 들어서 개봉하자마자 감탄만 연발했네요~ 색상도 용달차 색상이라는 말 때문에 잠시 고민했었는데ㅎㅎ 전~혀 아니예요~ 훨씬 깊이있고 진한 네이비 느낌이구요 실제로 보면 더 고급스럽고 예쁩니다~ 12나올때까지 기다린 보람이 있네요~ 64기가 쓰면서 용량때문에 불편했어서 256기가로 선택했는데 그것도 잘 선택한거 같구요~ 오래오래 잘 쓰려고 보호 필름이랑 케이스 주문할 때까지 손 안대고 기다렸네요~ ~~!!^^ 평점5 

4 역시 ㅠㅠㅠ 너무 이쁩니다실버와 그래파이트 둘중 계속 고민하고 주문했다 취소하고를 반복한 끝에

<ipython-input-21-f124c4fc7899>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


페이지 2 

21 아이폰을 사고 배송받은 뒤 며칠 사용해본 후에 리뷰 작성하고 싶어서 이제서야 후기를 씁니다ㅎㅎ🍎구매계기사전예약은 커녕 출시된 이후에도 재고 기다리는 사람들을 보고 이번 시리즈 사
아이폰을 사고 배송받은 뒤 며칠 사용해본 후에 리뷰 작성하고 싶어서 이제서야 후기를 씁니다ㅎㅎ

🍎구매계기
사전예약은 커녕 출시된 이후에도 재고 기다리는 사람들을 보고 이번 시리즈 사는 건 포기해야겠다고 생각했어요.카드 행사나 적립금 하나 없이 제 가격 다 주고 사는 건 싫고 핸드폰이 갖고 싶기는 하고 그래서 이리저리 서치하다가 찾은 것이 네이버 애플 공식 판매점입니다.공식이라는 설명에 마음이 갔고 여러 방법 다 하면 가격도 십만원 정도 싸게 살 수 있다는 것이 가장 .

🍎재고
제가 사려고 알아보던 때에는 모든 색상이 다 품절이었어요.사전예약도 아니고 조금 더 기다리면 언젠가는 들어오겠지라는 생각으로 매일 들어와서 재고확인 했던 것 같아요.오전 10시~11시쯤에 한번 물량 풀리고 오후 5시쯤에 또 한번 풀렸었는데 그파랑 실버,골드 다 들어오는데 퍼블만 재고가 안 들어와서 며칠 더 기다렸어요ㅠ지금은 모든 상품이 다 품절이 안되서 원활하게 구매하실 수 있으실 것 같네요!

🍎색상
퍼시픽 블루가 출시된다는 소식을 들었을 때부터 이번 핸드폰은 이거다 라고 생각했어요.근데 막상 구매까지 시간이 좀 걸리는 도중에 보니까 실버가 구하기 힘들다고 하니까 그것도 탐나서 이리저리 헤매다가 결국은 다시 블루로 돌아와 구매했습니다.구매하고 나서도 질리면 어쩌지 고민 많이 했는데 직접 보는 순간 그런 생각이 다 사라졌습니다..영상이랑 사진 전부 실물을 못 담아요..프로 자체가 무광이라 보는 각도마다 그파로 보이기도 하고 네이비로도 보였다가 블루로도 보여요!퍼블 사고 질리면 어떡하냐는 말을 제일 많이 들었는데 실버도 계속 실버만 보면 질리고 그파도 그파만 보면 질리듯이 똑같은 것 같습니다!

🍎배송
후기 보면서 배송이 엄청 늦었다, 타지역으로 상품이 가서 별로라는 글이 꽤 있었는데 제가 

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name_=name[1]  #z플
category_=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-22-dfb70b2f5d91>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-22-dfb70b2f5d91>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-22-dfb70b2f5d91>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-22-dfb70b2f5d91>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-22-dfb70b2f5d91>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 아이폰 xr을 사용하다가 아이폰 12pro로 넘어왔습니다 크기는 12pro가 xr보다 조금 작지만 베젤이 줄어들아 화면 크기는 비슷합니다 옆 테두리부분이 각지고 알루미늄 재질이라
아이폰 xr을 사용하다가 아이폰 12pro로 넘어왔습니다
크기는 12pro가 xr보다 조금 작지만 베젤이 줄어들아 화면 크기는 비슷합니다
옆 테두리부분이 각지고 알루미늄 재질이라 예쁘고 새로 나온 퍼시픽블루 색상을 한달 사용했는데 아직 질리지 않네요
그리고 터치감이나 반응속도가 조금 더 빨라진게 느껴집니다
배터리 시간은 비슷한거 같은데 무선 충전이 가능해서 편리하네요
아주 큰 변화는 없지만 저는 전체적으로 평점5 



<ipython-input-22-dfb70b2f5d91>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


2 아이폰6-8플러스-12프로 짝수에디션 사용자가 됐네요ㅎㅎ 8플러스 아직 너무 멀쩡하고 잘 쓰고 있었는데 크기가 너무 컸어서 갈아타게 됐습니다ㅎㅎ 그립톡 없이도 폰을 들 수 있다는게
아이폰6-8플러스-12프로 짝수에디션 사용자가 됐네요ㅎㅎ 8플러스 아직 너무 멀쩡하고 잘 쓰고 있었는데 크기가 너무 컸어서 갈아타게 됐습니다ㅎㅎ 그립톡 없이도 폰을 들 수 있다는게 참 신기하네요🤣 카메라 화질은 딱 보면 차이가 날 정도이고 나머지는 자급제로 쓰다보니 lte 사용해서 체감속도차이는 몰
겠어요ㅎㅎ 오프라인으로 모든 색상보고 골드가 가장 예뻐서 골드로 갔고요 원래는 실버만 사용했었습니다ㅎㅎ 다만 터치아이디 익숙하신 분들은 처음에 적응안될수도 있다 생각합니다! 저는 패드가 페이스 아이디라 그나마 금방 적응했고 적응안되는건 상단바에 배터리잔량표시 안되는거네요ㅎㅎ 아직까지는 뽑기를 잘했는지 이주정도 사용했는데 큰 이상없이 잘 사용하고 있습니다! 평점5 

3 최고예요
장점 : 카메라가 좋다. 크기가 적당하다. 빠릿하다. 고급시랍다. 스피커 소리가 좋다. 이쁘다.단점 : 비싸다. 무겁다. 많이 사용하면 뜨거워진다.배송은 2일만에 왔습니다.기스도 하나 없고 상태 좋네요. 비싼거라 케이스도 미리 시켜서 껴놓으니 맘이 놓이네요. 평점5 

4 배송 빠르고 영롱하고 예쁘네요!! 아이폰 8에서 넘어왔는데 확실히 사이즈는 커서 그립감이 다르지만 그외 다 만족합니다!!
배송 빠르고 영롱하고 예쁘네요!! 아이폰 8에서 넘어왔는데 확실히 사이즈는 커서 그립감이 다르지만 그외 다 만족합니다!! 평점5 

5 프로 맥스 샀다가 크기로 인해 휴대가 불편하여 다시 샀어요~ 역시 남자는 이 사이즈가 딱입니다. 배송도 빨랐네요.
프로 맥스 샀다가 크기로 인해 휴대가 불편하여 다시 샀어요~ 역시 남자는 이 사이즈가 딱입니다. 배송도 빨랐네요. 평점5 



<ipython-input-22-dfb70b2f5d91>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category6).click() #스크롤 건드리면 안됨
name_=name[1]  #z플
category_=category[5]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-23-5b32ff48846a>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-23-5b32ff48846a>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-23-5b32ff48846a>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category6).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-23-5b32ff48846a>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-23-5b32ff48846a>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 너무 좋아요. 한달쯤 사용중인데튼튼하고 깔끔하고 변색도없고 최고에요.요런 투명케이스들 변색심하고 얇거나 튼튼하면 무겁던데 요건 변색도없고 가볍고 튼튼해요. 그립감도 좋고 미끄럽지도
너무 좋아요. 한달쯤 사용중인데
튼튼하고 깔끔하고 변색도없고 최고에요.
요런 투명케이스들 변색심하고 얇거나 튼튼하면 무겁던데 요건 변색도없고 가볍고 튼튼해요. 그립감도 좋고 미끄럽지도 뻣뻣하지도 않아서 딱이에요.
투명이라서 제맘대로 스티커, 스마트톡 등으로 디자인할수있어서 더좋아요. 구매하세요. 평점5 



<ipython-input-23-5b32ff48846a>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


2 포장은 뽁뽁이 한겹싸져있고 박스는 두껍고 튼튼하긴 했어요 폰 키자마자 불량테스트랑 오줌 액정인지 확인 먼저 했는데 다행이 양품인것같지만 위아래 모서리에 유격 현상이 있네요 종이가
포장은 뽁뽁이 한겹싸져있고 박스는 두껍고 튼튼하긴 했어요
폰 키자마자 불량테스트랑 오줌 액정인지 확인 먼저 했는데 다행이 양품인것같지만
위아래 모서리에 유격 현상이 있네요 종이가 잘 들어간다는..

+ 대한통운 배송은 진짜 너무 느려요
다른상품 배송 됐는데 이것만 며칠 군포에 있다가
겨우 받음 대한통운 안썼으면.. 평점3 

3 품질좋고 최고예요~
품질좋고 최고예요~ 평점5 



<ipython-input-23-5b32ff48846a>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category7).click() #스크롤 건드리면 안됨
name_=name[1]  #z플
category_=category[6]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-24-f563be50f8eb>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-24-f563be50f8eb>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-24-f563be50f8eb>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category7).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-24-f563be50f8eb>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-24-f563be50f8eb>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 음량 버튼 누르는 곳에 스크래치 있는게 너무 신경쓰여서 서비스센터 가서 코스메틱 이슈로 교환 받으려 했더니 이곳에서 애플케어 플러스 신청해서 제품 받은 것 때문에 제품 출발때부터
음량 버튼 누르는 곳에 스크래치 있는게 너무 신경쓰여서 서비스센터 가서 코스메틱 이슈로 교환 받으려 했더니 이곳에서 애플케어 플러스 신청해서 제품 받은 것 때문에 제품 출발때부터 +1일씩 자동 계산돼서 교환기간 끝났다고 못해준다고 해서 쉬는 날 짜증 한 가득 안고 집에 왔네요.. 정신승리하고 케이스 끼고 쓰다보니 안보여서 상관없는데 그날만 생각하면 아 여기서 괜히 샀다.. 했네요 진짜.. 이것저것 거슬리시는 분들은 안정적으로 공홈에서 사세요. 저는 여기서 포인트랑 이것저것 할인 받은 거 생각하면 그거 값이다 생각하고 지금은 또 잘 쓰고 있습니다.
실버 테두리 지문 엄청 묻고 먼지 닿아도 잔스크래치 생긴다니까 실버 사실 분들은 참고하세요~~ 그리고 택배는 진짜 그냥 일반물품 오듯이 옵니다ㄷㄷ 평점4 



<ipython-input-24-f563be50f8eb>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


2 사전예약으로 구매해서 처음부터 매우 잘 쓰고 있어요. 폰 자체가 워낙 좋아요. 다만 그냥 폰 특성상 지문이 잘묻는게 단점이고, 다른 점은 만족스러워요! 임튼 자급제 폰을 네이버에서
사전예약으로 구매해서 처음부터 매우 잘 쓰고 있어요. 폰 자체가 워낙 좋아요. 다만 그냥 폰 특성상 지문이 잘묻는게 단점이고, 다른 점은 만족스러워요! 임튼 자급제 폰을 네이버에서 살 수 있어서 매우 만족입니다! 앞으로도 자급제 폰은 네이버에서 구매할까 생각중입니다! 할인만 좀 더 있으면 좋겠네요 ㅋㅋ 평점5 



<ipython-input-24-f563be50f8eb>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category8).click() #스크롤 건드리면 안됨
name_=name[1]  #z플
category_=category[7]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-25-30bc1c969459>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-25-30bc1c969459>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-25-30bc1c969459>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category8).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-25-30bc1c969459>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-25-30bc1c969459>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 배송 이틀만에 왓어요 색상 실버삿는데 화이트갓아요 넘이뻐여 심멎~~ 아직 케이스가없어서 사용못하고 잇네요
배송 이틀만에 왓어요 색상 실버삿는데 화이트갓아요 넘이뻐여 심멎~~ 아직 케이스가없어서 사용못하고 잇네요 평점5 



<ipython-input-25-30bc1c969459>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


2 잠깐 위메프 디지털데이 특가 나왔을때 운좋게 샀네요. 빨리 왔고 수능본 딸네미가 원하던 색상이고 충전기가 없는건 좀 황당하네요...
잠깐 위메프 디지털데이 특가 나왔을때 운좋게 샀네요. 빨리 왔고 수능본 딸네미가 원하던 색상이고 충전기가 없는건 좀 황당하네요. 애플넘들... 디자인은 그래도 왜 사야하는지를 말해주네요 ㅋ 평점5 



<ipython-input-25-30bc1c969459>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category9).click() #스크롤 건드리면 안됨
name_=name[1]  #z플
category_=category[8]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-26-608931ea7ce9>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-26-608931ea7ce9>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-26-608931ea7ce9>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category9).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-26-608931ea7ce9>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-26-608931ea7ce9>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 너무 좋아요. 한달쯤 사용중인데튼튼하고 깔끔하고 변색도없고 최고에요.요런 투명케이스들 변색심하고 얇거나 튼튼하면 무겁던데 요건 변색도없고 가볍고 튼튼해요. 그립감도 좋고 미끄럽지도
너무 좋아요. 한달쯤 사용중인데
튼튼하고 깔끔하고 변색도없고 최고에요.
요런 투명케이스들 변색심하고 얇거나 튼튼하면 무겁던데 요건 변색도없고 가볍고 튼튼해요. 그립감도 좋고 미끄럽지도 뻣뻣하지도 않아서 딱이에요.
투명이라서 제맘대로 스티커, 스마트톡 등으로 디자인할수있어서 더좋아요. 구매하세요. 평점5 



<ipython-input-26-608931ea7ce9>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


2 쓰던 폰이 고장나서 빨리 받을 수 있는 곳을 원했는데 배송이 너무 빨랐어요! 토욜에 시켰는데 화요일 오전에 왔거든요. 오히려 폰케이스랑 액정이 늦게 도착해서 기다림,,12시리즈는
쓰던 폰이 고장나서 빨리 받을 수 있는 곳을 원했는데 배송이 너무 빨랐어요! 토욜에 시켰는데 화요일 오전에 왔거든요. 오히려 폰케이스랑 액정이 늦게 도착해서 기다림,,
12시리즈는 다들 불량검사?를 하길래 저도 해봤는데 다 정상이었어요! 다들 폰 무겁다는데 전에 쓰던 폰보다 커서 무게가 더 나가는게 이상하지 않아요
너무 이뻐서 폰 볼때마다 기분이 좋아요 폰 잘쓸게요!
그리고 다들 폰사진 찍어올리던데 뭘로 찍어서 올리는 건가요? 저는 박스밖에 못찍었어요 평점5 

3 색상이 생각했던것보다 훨씬 마음에 드네요 예전에 쓰던 7+보다 가벼워서 좋습니다
색상이 생각했던것보다 훨씬 마음에 드네요 예전에 쓰던 7+보다 가벼워서 좋습니다 평점5 

4 실버 역시 너무 예뻐요 ㅠㅠ아이폰 8쓰다가 쓰는거라 무게는 좀 더 무겁긴 하지만 너무 예쁘고 좋아요. 용량도 넉넉하구요 ㅎㅎ
실버 역시 너무 예뻐요 ㅠㅠ
아이폰 8쓰다가 쓰는거라 무게는 좀 더 무겁긴 하지만 너무 예쁘고 좋아요. 용량도 넉넉하구요 ㅎㅎ 평점5 



<ipython-input-26-608931ea7ce9>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category10).click() #스크롤 건드리면 안됨
name_=name[1]  #z플
category_=category[9]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-27-1dd67d7950c6>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-27-1dd67d7950c6>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-27-1dd67d7950c6>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category10).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-27-1dd67d7950c6>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-27-1dd67d7950c6>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 음량 버튼 누르는 곳에 스크래치 있는게 너무 신경쓰여서 서비스센터 가서 코스메틱 이슈로 교환 받으려 했더니 이곳에서 애플케어 플러스 신청해서 제품 받은 것 때문에 제품 출발때부터
음량 버튼 누르는 곳에 스크래치 있는게 너무 신경쓰여서 서비스센터 가서 코스메틱 이슈로 교환 받으려 했더니 이곳에서 애플케어 플러스 신청해서 제품 받은 것 때문에 제품 출발때부터 +1일씩 자동 계산돼서 교환기간 끝났다고 못해준다고 해서 쉬는 날 짜증 한 가득 안고 집에 왔네요.. 정신승리하고 케이스 끼고 쓰다보니 안보여서 상관없는데 그날만 생각하면 아 여기서 괜히 샀다.. 했네요 진짜.. 이것저것 거슬리시는 분들은 안정적으로 공홈에서 사세요. 저는 여기서 포인트랑 이것저것 할인 받은 거 생각하면 그거 값이다 생각하고 지금은 또 잘 쓰고 있습니다.
실버 테두리 지문 엄청 묻고 먼지 닿아도 잔스크래치 생긴다니까 실버 사실 분들은 참고하세요~~ 그리고 택배는 진짜 그냥 일반물품 오듯이 옵니다ㄷㄷ 평점4 



<ipython-input-27-1dd67d7950c6>:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')


2 사려고 거의 3주 동안 들락날락 했던 것 같아요. 처음엔 그렇게 절박하게 하지 않았다가 본격적으로 사려고 10시~11시 사이랑 5시~6시 사이에 새로고침 많이 했습니다. 퍼시픽 블
사려고 거의 3주 동안 들락날락 했던 것 같아요. 처음엔 그렇게 절박하게 하지 않았다가 본격적으로 사려고 10시~11시 사이랑 5시~6시 사이에 새로고침 많이 했습니다. 퍼시픽 블루가 가장 많이 들어왔고 그 다음으로는 그래파이트랑 골드가 많이 들어왔어요. 저는 실버를 원했는데 실버는 잘 안 들어오더라구요. 문의글에서 실버가 이번주나 다음주에 올라온다는 얘기를 보고 며칠동안 하다가 저녁 6시가 다 되서 실버가 떴길래 샀습니다. 배송은 2일 걸렸어요. 포장은 애플 박스를 뽁뽁이로 한 번 두른 상태였는데 이 포장된 박스를 고정 시킬 수 있는 장치가 있거나 택배 박스가 딱 맞는 사이즈면 더 좋을 것 같다고 생각했습니다. 열자마자 핸드폰 외관을 꼼꼼히 살펴봤는데 음량 버튼 밑에 기스랑 오른쪽 하단 부분에 잔기스가 있었습니다. 그리고 그날 날씨가 많이 추웠는데 아마 한파주의보였던 것 같아요. 그래서 폰에 무리가 갔을까봐 걱정했는데 기본적인 성능 확인해보니 정상이라고 뜨더라구요. 디스플레이도 문제 없었구요. 기스 많이 티나진 않았지만 아무래도 신경쓰여서 문의드렸더니 애플 공식 서비스센터 가보라고 하시더라구요. 그래서 다음날 바로 갔습니다. 교환 안될까봐 걱정했는데 확인해보시더니 교환 진행해주셨습니다. 다행히 재고가 있어서 바로 교환하고 왔네요. 요즘엔 재고가 좀 많이 풀리는 것 같아요. 말이 길었지만...여기서 구매해서 9만원대 정도의 적립금도 받고 초반 배송도 빨라서 좋았습니다^^ 외관 기스는 여기서도 어쩔 수 없는 일이니...전반적으로 . 받아보니 질리지 않을 색상이네요. 평점5 

3 배송느려요 ㅠㅠ후기 보니깐 다들 빨리 받았다고 하는데...화요일에 주문하고 토요일에 받았어요. 기다리느라 숨넘어가는줄 ;; ㅠㅠ다행히 제품하자는 없어요. 마감 불량도 없고 좋아요~
배송느려요 ㅠㅠ
후기 보

<ipython-input-27-1dd67d7950c6>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category11).click() #스크롤 건드리면 안됨
name_=name[1]  #z플
category_=category[10]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-28-b5706be7cbf2>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-28-b5706be7cbf2>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-28-b5706be7cbf2>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category11).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-28-b5706be7cbf2>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-28-b5706be7cbf2>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 그래파이트랑 고민하다가 실버샀는데 존녜에여진짜ㅠㅠ 전에7쓰던거에 비하면 많이 무겁긴 하지만 배터리 오래가고 화질 좋아서 안고갑니다
그래파이트랑 고민하다가 실버샀는데 존녜에여진짜ㅠㅠ 전에7쓰던거에 비하면 많이 무겁긴 하지만 배터리 오래가고 화질 좋아서 안고갑니다 평점5 



<ipython-input-28-b5706be7cbf2>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_iphone)
req = requests.get(ns_address_iphone)
html = req.text 
soup = BeautifulSoup(html, "html.parser")


#"디자인" 관련 리뷰 가져오기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)
d.find_element_by_xpath(category12).click() #스크롤 건드리면 안됨
name_=name[1]  #z플
category_=category[11]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
       
        sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생  
            next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


<ipython-input-29-76ec3f8da5f8>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C://Users//New//Desktop//Project//chromedriver.exe') # webdriver = chrome
<ipython-input-29-76ec3f8da5f8>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(shoppingmall_review).click()
<ipython-input-29-76ec3f8da5f8>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  d.find_element_by_xpath(category12).click() #스크롤 건드리면 안됨


페이지 1 



<ipython-input-29-76ec3f8da5f8>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  star=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[1]/span[1]').text
<ipython-input-29-76ec3f8da5f8>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text


1 실버가 실물이 너무너무 예쁘고 마지막 재고였을 때 구매해서 많이 안기다리고 좋았어요애플케어도 할인해서 가입하고 핸드폰도 네이버적립해서 자급제인데 꽤 저렴하게 구매했어요근데 뽑기 실
실버가 실물이 너무너무 예쁘고 마지막 재고였을 때 구매해서 많이 안기다리고 좋았어요\n애플케어도 할인해서 가입하고 핸드폰도 네이버적립해서 자급제인데 꽤 저렴하게 구매했어요\n\n근데 뽑기 실패인가봐요..\n검색하거나 메세지 쓸 때 키보드가 안올라와서 재부팅하고, 데이터랑 와이파이가 잘 안잡히고 어느순간은 아예 연결불가라 재부팅하고, 며칠전이랑 오늘은 베터리 남아있는상태로 자고 일어났는데 핸드폰이 방전됐고 결국 오늘 알람이 안울려서 지각했어요ㅠㅠ 평점2 



<ipython-input-29-76ec3f8da5f8>:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page=d.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click()


In [ ]:
#df3 = pd.read_csv('C://Users//New//Desktop//Project//output_iphone_total.csv')
#df3['star'] = df3['star'].str.replace(pat=r'[ㄱ-ㅣ가-힣]+', repl= r'', regex=True)
#df3.drop(df3.columns[[0]], axis=1)
#df3.to_csv('output_iphone_Total(1).csv', encoding='utf-8-sig', mode='w')